In [ ]:
import sys
import os
import pandas as pd
pd.set_option('display.max_columns', 30)

sys.path.insert(0, '../')
from data_collection import (
    PlanDataCollector,       # raw 데이터 수집
    collect_click_logs,      # 클릭로그 수집
    attach_ctr_label,        # CTR annotation
    PlanDataRefiner,         # 데이터 정제
    train_valid_test_split   # 학습/검증/테스트 데이터 split
)

# Plan Data Collection

*  기획전 raw 데이터 수집
* 데이터 수집 후 디렉토리 구성 (`CORE`: 학습에 직접적으로 활용됨)
```shell
data/                      # 수집 후 기본 디렉토리명: '{START_DATE}_{END_DATE}' (ex. '2022-01-22_2022-01-23')
├plan_data/                # 기획전 관련 데이터
│  ├plan_section_meta/     # 각 기획전 내 섹션별 메타 데이터
│  └plan_data_all.csv      # 기획전 메타 데이터
├prod_data/                # 상품 관련 데이터
│  ├images/                # (CORE) 상품 이미지 디렉토리
│  ├prod_img_meta/         # 이미지 URL 등 상품 이미지 메타 데이터 디렉토리 
│  ├prod_ocr_meta/         # 상품 이미지 내 OCR 데이터 디렉토리
│  ├prod_data_all.csv      # 상품 메타 데이터
│  └prod_section_names.pkl # 상품의 소속 섹션명 정보
└raw.csv                   # (CORE) 학습용 raw 데이터
```

In [ ]:
# 기획전 수집 기간 (노출일 기준 필터링)
start_date = "2021-12-30"
end_date = "2021-12-31"

# raw 데이터 저장 디렉토리
save_dir = "./test/"

# True: 상품 이미지를 리사이즈(512x512)하여 수집 (False: 원본 크기로 저장)
pre_resize = True

In [ ]:
# 참고: 삭제/판매중지 등의 이슈로 데이터를 온전히 수집할 수 없는 상품은 버림
collector = PlanDataCollector()
raw = collector.collect(
    start_date,             # 수집 시작일
    end_date,               # 수집 말일
    save_dir,               # raw 데이터 저장 디렉토리
    pre_resize=pre_resize,  # True: 이미지 저장 시 512x512로 일괄 리사이즈
    return_data=True        # True: raw 데이터를 저장 후 해당 데이터를 곧바로 return
)

In [ ]:
raw.sample()

# Click Logs Collection & CTR Annotation
* 클릭 로그 수집 및 `Plan Data Collection`에서 얻은 `raw` 데이터에 CTR 어노테이션
* ***NOTE.*** 클릭 로그 수집을 위해서는 데이터 접근 권한 key가 필요

In [ ]:
# 클릭 로그 수집 기간
start_date = "2021-12-30"
end_date = "2021-12-31"

# key
key = {
    'location': '[LOCATION]',
    'ticket': '[YOUR_TICKET]',
    'principal': '[PRINCIPAL_INFO]',
    'keytab_file': '[KEYTAB_FILE_PATH]',
    'config': '[CONFIG_INFO]'
}

# 저장 디렉토리
save_dir = "./test_click_logs/"

In [ ]:
click_logs = collect_click_logs(
    start_date=start_date,   # 수집 시작일
    end_date=end_date,       # 수집 말일
    key=key,                 # 데이터 액세스를 위한 key 정보
    save_dir=save_dir,       # 수집한 데이터 저장 디렉토리
    return_data=True         # True: 클릭 로그 데이터 저장 후 해당 데이터를 곧바로 return
)
click_logs.head()

### `ctr` Annotation

In [ ]:
raw_labeled = attach_ctr_label(raw, click_logs=click_logs)

# Data Refinement
* CLIK 학습에 맞는 학습 데이터 구성
* target 및 기타 설정값에 따라 데이터를 정제함
* 타깃으로 활용할 만한 컬럼
  * `prod_review_cnt`: 리뷰수
  * `prod_zzim_cnt`: 찜수
  * `prod_sale_cnt`: 판매수
  * `ctr`: CTR
  
* 정제 절차
  * I. (optional) 각 기획전으로부터 `K`개 상품을 샘플링
  * II. `target`이(가) 0보다 큰 상품이 `N`개 이상인 기획전만 추출
  * III. `target`에 대한 uniqueness가 `M` 이상인 기획전만 추출

In [ ]:
refine_rule = dict(
    target='ctr',          # 학습에 활용할 타깃컬럼 ('ctr', 'prod_review_cnt', ...)
    sample_size=None,      # 각 기획전으로부터 {sample_size}개의 상품을 샘플링 (None: 무제한)
    min_nonzero_num=5,     # target이 non-zero인 상품이 {min_nonzero_num}개 이상인 기획전만 필터링
    unique_constraint=2,   # target의 uniqueness가 {unique_constraint} 이하 기획전 제거
)
seed = 27                  # 랜덤시드

refiner = PlanDataRefiner(
    rule=refine_rule,      # 정제 규칙
    seed=seed              # 랜덤시드
)

In [ ]:
raw_refined = refiner.sift(
    raw_labeled, 
    verbose=True # True: 정제 중 카테고리별 빈도 출력
)

* (참고) 학습 데이터는 다음과 같은 데이터프레임을 활용함

In [ ]:
columns = [
    "plan_id",          # 기획전 ID
    "plan_name",        # 기획전명
    "plan_startdate",   # 기획전 게시일자
    "plan_cat1",        # 기획전 카테고리(depth=1)
    "plan_cat2",        # 기획전 카테고리(depth=2)
    "plan_kwds",        # 기획전 키워드
    "prod_name",        # 상품명
    "prod_text",        # 상품상세
    "prod_opendate",    # 상품 게시일자
    "prod_cat1",        # 상품 카테고리(depth=1)
    "prod_cat2",        # 상품 카테고리(depth=2)
    "prod_cat3",        # 상품 카테고리(depth=3)
    "prod_cat4",        # 상품 카테고리(depth=4)
    "prod_page_title",  # 상품 페이지명
    "ctr"               # 타깃컬럼
]
raw_refined[columns].sample()

# Train Valid Test Split
* 정제를 마친 뒤 학습/검증/테스트 데이터 split

In [ ]:
train_size = 0.7     # 학습 데이터 비율
valid_size = 0.2     # 검증 데이터 비율
test_size = 0.1      # 테스트 데이터 비율 (0으로 설정 시 학습/검증 데이터로만 분리됨)
eval_cutoff = True   # True: 검증/테스트 내 기획전의 상품을 50개로 한정 (샘플링으로 인한 가변적인 평가 방지)
target = 'ctr'       # 타깃컬럼
deleakage = True     # True: 학습 <-> 검증/테스트 간 leakage 제거
seed = 27            # 랜덤시드

train, valid, test = train_valid_test_split(
    meta=raw_refined, 
    train_size=train_size, 
    valid_size=valid_size, 
    test_size=test_size,
    eval_cutoff=eval_cutoff,
    target=target,
    deleakage=deleakage,
    random_state=seed
)

* 최종 정제 학습/검증/테스트 데이터 저장

In [ ]:
save_dir = "./test/2021-12-30_2021-12-31/"
train.to_csv(os.path.join(save_dir, "train.csv"), index=False)
valid.to_csv(os.path.join(save_dir, "valid.csv"), index=False)
test.to_csv(os.path.join(save_dir, "test.csv"), index=False)